In [1]:
from elastic_db.elasticsearch import elastic_conection, NLPmodelIndex

es = elastic_conection()
index = NLPmodelIndex(es=es, workspace_id="dc1e7b3d-9137-4a20-a99c-d0d2029ef170-cedae")
index.index_name

'nlp_model-dc1e7b3d-9137-4a20-a99c-d0d2029ef170-cedae-02022021092147'

# Function to talk with Omni NLU model

In [16]:
from tensorflow.keras.models import load_model
import pandas as pd
import pickle
import json

with open("data/intents_dictionary_tfidf_cedae.json", 'r') as f:
      intents_dictionary = json.load(f)

omni_model = load_model('nlp_models/cedae-bestModel-70.75acc-2020-02-01.h5')
tf_idf_corpus = pickle.load(open("nlp_models/tf_idf_corpus_cedae.pickle", "rb"))
df_test = pd.read_csv("data/test_data_cedae.csv", sep=",")
df_test.head()

,example,intent
0,quero ir numa agência da CEDAE,Localizar_AgenciasCEDAE
1,Boa Noite,General_Greetings
2,quero um relatório dos meus pagamentos,Consultar_ContasPagas
3,você sabe falar sobre o que,Bot_Capabilities
4,cobranca indevida,Revisar_Conta


In [3]:
import numpy as np

query = {
  "tokenizer" : "classic",
  "filter" : [
              "lowercase",
              "asciifolding",
              {"type": "stop", "stopwords": "_portuguese_"},
              {"type": "stemmer", "language": "brazilian"}],
  "text" : ""
}

def get_sentece_features(sentence):
    query["text"] = sentence
    result = es.indices.analyze(index=index.index_name, body=query)
    new_sentence = " ".join([token["token"] for token in result["tokens"]])
    sentence_features = tf_idf_corpus.transform([new_sentence]).toarray()
    return sentence_features 

def omni_predict(sentence):
    sentence_features = get_sentece_features(sentence)
    result = omni_model.predict(sentence_features)
    idx = np.argmax(result)
    intent, score = intents_dictionary[str(idx)], result[0][idx]
    return intent, score

In [18]:
sentence = "quero ir numa agência da CEDAE"
omni_predict(sentence)

('Localizar_AgenciasCEDAE', 0.9998635)

# Function to talk with Watson Model

In [7]:
import os

API_KEY = "nA36HqlqQkQSDf0EywIeYGPl0qI_nNvLqvBUI_zhhUmR" # os.environ['API_KEY']
SERVICE_URL = "https://api.us-south.assistant.watson.cloud.ibm.com/instances/12c017ac-1bb4-4364-8493-9cc9ba98aa26" # os.environ['SERVICE_URL']
ASSISTANT_ID = "eb61f20c-beb4-491f-a9e4-b36c65bcae9c" # os.environ['ASSISTANT_ID']

In [8]:
import json
from ibm_watson import AssistantV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator(API_KEY)
assistant = AssistantV1(
    version='2020-04-01',
    authenticator = authenticator
)

assistant.set_service_url(SERVICE_URL)

def watson_predict(sentence):
    response = assistant.message(
        workspace_id=ASSISTANT_ID,
        input={
            'message_type': 'text',
            'text': sentence
        }
    ).get_result()
    if response.get("intents") and response["intents"]:
        return (response["intents"][0]["intent"], response["intents"][0]["confidence"])
    else:
        return "failed to identify", -1.0

In [9]:
watson_predict(sentence)

('Localizar_AgenciasCEDAE', 0.9050686836242676)

# Battle Omni NLP API vs. Watson Assistent 

In [19]:
omni_right_answer = 0
watson_right_answer = 0
for _, row in df_test.iterrows():
    omni_result = omni_predict(row.example)
    watson_result = watson_predict(row.example)
    
    print("-------------------------------------------")
    print("sentence: %s"%row.example)
    print("\n")
    print("Omni NLP API\n")
    print("predict_intent: %s"%omni_result[0])
    print("score: %.2f"%omni_result[1])
    print("right_intent: %s"%row.intent)
    if omni_result[0] == row.intent:
        omni_right_answer += 1
        print("is it right?: yes")
    else:
         print("is it right?: no")
    print("quantity of right answers: %s" %omni_right_answer)
    print("\n")
    
    print("Watson Assistent\n")
    print("predict_intent: %s"%watson_result[0])
    print("score: %.2f"%watson_result[1])
    print("right_intent: %s"%row.intent)
    if watson_result[0] == row.intent:
        watson_right_answer += 1
        print("is it right?: yes")
    else:
         print("is it right?: no")
    print("quantity of right answers: %s" %watson_right_answer)
    print("\n")

print("-------------------------------------------\n")
print("Total of questions: %d\n"%df_test.shape[0])
print("Omni NLP API\n") 
print("Total of right answers: %d"%omni_right_answer)
omni_acc = round(omni_right_answer/df_test.shape[0], 2)*100
print("Accuracy: %s\n"%omni_acc)

print("Watson Assistent\n") 
print("Total of right answers: %d"%watson_right_answer)
watson_acc = round(watson_right_answer/df_test.shape[0], 2)*100
print("Accuracy: %s"%watson_acc)

-------------------------------------------
sentence: quero ir numa agência da CEDAE


Omni NLP API

predict_intent: Localizar_AgenciasCEDAE
score: 1.00
right_intent: Localizar_AgenciasCEDAE
is it right?: yes
quantity of right answers: 1


Watson Assistent

predict_intent: Localizar_AgenciasCEDAE
score: 0.91
right_intent: Localizar_AgenciasCEDAE
is it right?: yes
quantity of right answers: 1


-------------------------------------------
sentence: Boa Noite


Omni NLP API

predict_intent: General_Ending
score: 0.22
right_intent: General_Greetings
is it right?: no
quantity of right answers: 1


Watson Assistent

predict_intent: failed to identify
score: -1.00
right_intent: General_Greetings
is it right?: no
quantity of right answers: 1


-------------------------------------------
sentence: quero um relatório dos meus pagamentos


Omni NLP API

predict_intent: Consultar_ContasPagas
score: 0.66
right_intent: Consultar_ContasPagas
is it right?: yes
quantity of right answers: 2


Watson Ass

-------------------------------------------
sentence: Desejo denunciar uma ligação irregular de água


Omni NLP API

predict_intent: Realizar_Denuncia
score: 0.99
right_intent: Realizar_Denuncia
is it right?: yes
quantity of right answers: 14


Watson Assistent

predict_intent: Realizar_Denuncia
score: 0.99
right_intent: Realizar_Denuncia
is it right?: yes
quantity of right answers: 16


-------------------------------------------
sentence: Quero saber se tenho dívidas com a CEDAE


Omni NLP API

predict_intent: Emitir_NadaConsta
score: 0.80
right_intent: Emitir_NadaConsta
is it right?: yes
quantity of right answers: 15


Watson Assistent

predict_intent: Emitir_NadaConsta
score: 0.59
right_intent: Emitir_NadaConsta
is it right?: yes
quantity of right answers: 17


-------------------------------------------
sentence: como faço para encontrar vocês?


Omni NLP API

predict_intent: Localizar_AgenciasCEDAE
score: 0.43
right_intent: Localizar_AgenciasCEDAE
is it right?: yes
quantity of ri

-------------------------------------------
sentence: Debito em meu nome


Omni NLP API

predict_intent: Bot_About
score: 0.74
right_intent: Emitir_NadaConsta
is it right?: no
quantity of right answers: 29


Watson Assistent

predict_intent: Parcelar_Conta
score: 0.53
right_intent: Emitir_NadaConsta
is it right?: no
quantity of right answers: 32


-------------------------------------------
sentence: obrigada


Omni NLP API

predict_intent: General_Positive_Feedback
score: 1.00
right_intent: General_Ending
is it right?: no
quantity of right answers: 29


Watson Assistent

predict_intent: failed to identify
score: -1.00
right_intent: General_Ending
is it right?: no
quantity of right answers: 32


-------------------------------------------
sentence: obrigado


Omni NLP API

predict_intent: General_Positive_Feedback
score: 1.00
right_intent: General_Ending
is it right?: no
quantity of right answers: 29


Watson Assistent

predict_intent: General_Ending
score: 0.67
right_intent: General_E

-------------------------------------------
sentence: Estou com uma conta atrasada como faço para negociar


Omni NLP API

predict_intent: Emitir_SegundaViaConta
score: 0.70
right_intent: Parcelar_Conta
is it right?: no
quantity of right answers: 42


Watson Assistent

predict_intent: Emitir_SegundaViaConta
score: 0.61
right_intent: Parcelar_Conta
is it right?: no
quantity of right answers: 47


-------------------------------------------
sentence: tenho um vazamento na conexão do meu relógio


Omni NLP API

predict_intent: Reportar_Vazamento
score: 0.81
right_intent: Reportar_Vazamento
is it right?: yes
quantity of right answers: 43


Watson Assistent

predict_intent: Reportar_Vazamento
score: 0.95
right_intent: Reportar_Vazamento
is it right?: yes
quantity of right answers: 48


-------------------------------------------
sentence: Esqueci o meu CPF


Omni NLP API

predict_intent: Consultar_MatriculaCPF
score: 0.91
right_intent: Consultar_MatriculaCPF
is it right?: yes
quantity of ri

-------------------------------------------
sentence: Como faço para falar com a Cedae?


Omni NLP API

predict_intent: Solicitar_Contato
score: 0.65
right_intent: Solicitar_Contato
is it right?: yes
quantity of right answers: 59


Watson Assistent

predict_intent: Solicitar_Contato
score: 0.68
right_intent: Solicitar_Contato
is it right?: yes
quantity of right answers: 64


-------------------------------------------
sentence: Você tem um nome?


Omni NLP API

predict_intent: Bot_About
score: 1.00
right_intent: Bot_About
is it right?: yes
quantity of right answers: 60


Watson Assistent

predict_intent: Bot_About
score: 0.76
right_intent: Bot_About
is it right?: yes
quantity of right answers: 65


-------------------------------------------
sentence: Eu que agradeço


Omni NLP API

predict_intent: General_Ending
score: 0.96
right_intent: General_Positive_Feedback
is it right?: no
quantity of right answers: 60


Watson Assistent

predict_intent: General_Ending
score: 0.50
right_intent: